In [ ]:
# Python信用评分卡建模
# https://zhuanlan.zhihu.com/p/69548218

In [9]:
import pandas
train=pandas.read_csv('GiveMeSomeCredit-cs-training.csv')
test=pandas.read_csv('GiveMeSomeCredit-cs-test.csv')

In [11]:
# 3 数据预处理
import numpy
from sklearn.ensemble import RandomForestRegressor
def set_missing(df):
    process_df = df.iloc[:,[5,0,1,2,3,4,6,7,8,9]]
    # 分成已知特征值和未知特征值两部分
    known = process_df[process_df['MonthlyIncome'].notnull()].values# 将表格转换为矩阵    
    # known = process_df[process_df['MonthlyIncome'].notnull()].as_matrix() # 将表格转换为矩阵
    unknown = process_df[process_df['MonthlyIncome'].isnull()].values
    # x为特征属性
    X = known[:,1:]
    # Y为结果标签值,即要预测的MonthlyIncome
    Y = known[:,0]
    rfr = RandomForestRegressor(random_state=0,n_estimators=200,max_depth=3,n_jobs=-1)
    rfr.fit(X,Y)
    predicted = rfr.predict(unknown[:,1:]).round(0)
    # 用得到的预测结果填补原始缺失数据
    df.loc[(df['MonthlyIncome'].isnull()),'MonthlyIncome'] = predicted
    return df
# train = set_missing(train)

In [13]:
# 特征量NumberOfDependts
train=train.dropna()
train.drop_duplicates()
train.to_csv('GiveMeSomeCredit-cs-training-MissingData.csv',index=False)
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 120269 entries, 0 to 149999
Data columns (total 12 columns):
 #   Column                                Non-Null Count   Dtype  
---  ------                                --------------   -----  
 0   ID                                    120269 non-null  int64  
 1   SeriousDlqin2yrs                      120269 non-null  int64  
 2   RevolvingUtilizationOfUnsecuredLines  120269 non-null  float64
 3   age                                   120269 non-null  int64  
 4   NumberOfTime30-59DaysPastDueNotWorse  120269 non-null  int64  
 5   DebtRatio                             120269 non-null  float64
 6   MonthlyIncome                         120269 non-null  float64
 7   NumberOfOpenCreditLinesAndLoans       120269 non-null  int64  
 8   NumberOfTimes90DaysLate               120269 non-null  int64  
 9   NumberRealEstateLoansOrLines          120269 non-null  int64  
 10  NumberOfTime60-89DaysPastDueNotWorse  120269 non-null  int64  
 11  

In [14]:
# 变量SeriousDlqin2yrs取反，处理后即好客户为1，违约客户为0
train['SeriousDlqin2yrs'] = 1-train['SeriousDlqin2yrs']

In [17]:
# （3）数据切分
from sklearn.model_selection import train_test_split
Y=train['SeriousDlqin2yrs']
X=train.iloc[:,1:]
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size = 0.3,random_state=0)
train = pandas.concat([Y_train,X_train],axis = 1)
test = pandas.concat([Y_test,X_test],axis = 1)


In [ ]:
# 4 探索性分析

In [18]:
# 四、变量选择
# 1 分箱处理（变量离散化）
import pandas as pd
def mono_bin(Y,X,n = 20):
    good = Y.sum()
    bad = Y.count() - good
    r = 0
    while np.abs(r) < 1:
        d1 = pd.DataFrame({'X':X,'Y':Y,'Bucket':pd.qcut(X,n)})
        d2 = d1.groupby(['Bucket'],as_index = True)
        r,p = stats.spearmanr(d2['X'].mean(),d2['Y'].mean()) # spearman秩相关系数是度量两个变量之间的统计相关性的指标
        n = n - 1
    d3 = pd.DataFrame(d2['X'].min(),columns=['min']) # 为什么是空表
    d3['min'] = d2['X'].min()
    d3['max'] = d2['X'].max()
    d3['sum'] = d2['Y'].sum()
    d3['total'] = d2['Y'].count()
    d3['rate'] = d2['Y'].mean()
    d3['goodattribute'] = d3['sum'] / good
    d3['badattribute'] = (d3['total'] - d3['sum']) / bad
    d3['woe'] = np.log(d3['goodattribute'] / d3['badattribute'])
    iv = ((d3['goodattribute'] - d3['badattribute']) * d3['woe']).sum()
    d4 = (d3.sort_index(by = 'min'))
    woe = list(d4['woe'].values)
    print(d4)
    print('-' * 30)
    cut = []
    cut.append(float('-inf'))
    for i in range(1,n+1):
        qua = X.quantile(i/(n+1)) # quantile求分位数，pos = (n+1)*p，n为数据的总个数，p为0-1之间的值
        cut.append(round(qua,4)) # round()方法返回浮点数x的四舍五入值。
    cut.append(float('inf'))
    return d4,iv,woe,cut

In [21]:
# # 使用最优分段对于数据集中的RevolvingUtilizationOfUnsecuredLines、age、DebtRatio和MonthlyIncome进行分类。
# import numpy as np
# dfx1,ivx1,woex1,cutx1 = mono_bin(train['SeriousDlqin2yrs'],train['RevolvingUtilizationOfUnsecuredLines'],n = 10) 
# dfx2,ivx2,woex2,cutx2 = mono_bin(train['SeriousDlqin2yrs'],train['age'],n = 20)
# dfx4,ivx4,woex4,cutx4 = mono_bin(train['SeriousDlqin2yrs'],train['DebtRatio'],n = 10)
# dfx5,ivx5,woex5,cutx5 = mono_bin(train['SeriousDlqin2yrs'],train['MonthlyIncome'],n = 10)

In [ ]:
# https://zhuanlan.zhihu.com/p/69548218